# Make movie for conclusion slide

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from glob import glob
import numpy as np
# Peak detection
from scipy.spatial import cKDTree as KDTree
import sys
import os
import tempfile
import hyperspy.api as hs
import skimage.io

In [ ]:
imgroot = 'experiment/Au-CeO2__200_CO_7_250'
atomsroot = imgroot + '_atoms'

In [ ]:
def namegenerator(imgroot, dataroot):
    for root, dirs, files in os.walk(imgroot):
        dirs.sort()
        files.sort()
        for f in files:
            fullname = os.path.join(root, f)
            f_noext, ext = os.path.splitext(fullname)
            if ext not in ['.dm4', '.png']:
                continue
            assert f_noext.startswith(imgroot)
            dataname = dataroot + f_noext[len(imgroot):] + '_atoms.npy'
            yield fullname, dataname

In [ ]:
names = namegenerator(imgroot, atomsroot)

In [ ]:
def get_image(f):
    if f.endswith('.dm4'):
        a = hs.load(f)
        image = a.data
    else:
        a = skimage.io.imread(f)
        image = skimage.color.rgb2gray(a)
    return image

In [ ]:
def get_points(f):
    atoms = np.load(f)
    return atoms

In [ ]:
def crop(image, atoms, xmin=0, xmax=None, ymin=0, ymax=None):
    shape = image.shape
    if xmax is None:
        xmax = shape[0]
    if ymax is None:
        ymax = shape[1]
    image = image[xmin:xmax,ymin:ymax]
    d = 5
    atoms = atoms[np.where(atoms[:,0] > xmin + d)]
    atoms = atoms[np.where(atoms[:,1] > ymin + d)]
    atoms = atoms[np.where(atoms[:,0] < xmax - d)]
    atoms = atoms[np.where(atoms[:,1] < ymax - d)]
    atoms = atoms - np.array((xmin, ymin))
    return image, atoms

In [ ]:
def makefigure(image, atoms, plotfile):
    #image, atoms = crop(image, atoms, xmin=500, xmax=1150, ymin=150, ymax=800)

    fig, ax_atoms = plt.subplots(1,1,figsize=(10,10))
    #im1 = ax_raw.imshow(image,cmap='gray')
    im3 = ax_atoms.imshow(image, cmap='gray')
    #mycolor = '#80FFFF'
    mycolor = '#FFFF00'
    ax_atoms.scatter(atoms[:,1], atoms[:,0], c=mycolor, marker='o', linewidth=2.0)
    ax_atoms.axis('off')
    plt.tight_layout()
    fig.savefig(plotfile, bbox_inches='tight')
    plt.close(fig)

In [ ]:
#with tempfile.TemporaryDirectory() as tmpdir:
tmpdirectory = tempfile.TemporaryDirectory()
tmpdir = tmpdirectory.name
filenames = os.path.join(tmpdir, 'files.txt')
with open(filenames, "wt") as f:
    print("Temporary folder:", tmpdir)
    for i, (fi, fd) in enumerate(names):
        image = get_image(fi)
        atoms = get_points(fd)
        atoms = atoms[atoms[:,0] > 70]
        print(fi)
        pngfile = 'a{:04d}.png'.format(i)
        plotfile = os.path.join(tmpdir, 'a{:04d}.jpg'.format(i))
        tempfile = os.path.join(tmpdir, 'tmp_'+pngfile)
        makefigure(image, atoms, tempfile)
        os.system("convert -trim -quality 100 {} {}".format(tempfile, plotfile))
        #os.remove(tempfile)
        f.write(plotfile + '\n')
        #if i == 25: break

The images are converted to a movie using the `convert` utility from ImageMagick.  Unfortunately, it produces a video that PowerPoint cannot play, so it has to be converted again by a video conversion utility (I use Wondershare for Mac).

In [ ]:
cmd = 'convert -delay 5 -quality 95 @{} movie_conclusion.m4v'.format(filenames)
print(cmd)
os.system(cmd)

In [ ]:
# del tmpdirectory